### Imports

In [28]:
import os
import re
import tqdm
import sys

import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error as mae

!{sys.executable} -m pip install scipy
from scipy.stats import f_oneway


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


# Evaluation

In [29]:
def cs(gt, pred, alpha):
    """
    Computes the Cumulative Score with given alpha

    :param gt:    Ground truths
    :param pred:  Predicted dates
    :param alpha: Acceptable error range in years

    :return: The CS with given alpha
    """
    count = 0
    for i in range(len(gt)):
        absolute_error = abs(gt[i] - pred[i])
        if absolute_error <= alpha:
            count += 1

    return count / len(gt) * 100

### K-fold cross-validation

In [30]:
def kfold_cv(model, features, labels, k, seed):
    """
    Performs stratisfied k-fold cross-validation.

    :param model:    The model to train
    :param features: Features to train and test the model with
    :param labels:   Labels of the features
    :param k:        Number of folds for the cross-validation
    :param seed:     Seed for splitting data into test/train sets

    :return: Mean and SD of MAE, CS with alpha = 25 and alpha = 0 years across all folds.
    """
    mae_k = []
    cs_k = []
    cs_1 = []
    # As the dataset is imbalanced --> stratified kfold + seed to get the same validation/train splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)

    for train_index, test_index in kf.split(features, labels):
        # Getting test and train sets
        train_features, test_features = features[train_index], features[test_index]
        train_labels, test_labels = [labels[i] for i in train_index], [labels[i] for i in test_index]

        # Training model and predicting dates on test data
        model.fit(train_features, train_labels)
        pred = model.predict(test_features)
        pred = [int(i) for i in pred]

        mae_k.append(mae(test_labels, pred))
        cs_k.append(cs(test_labels, pred, 25))
        cs_1.append(cs(test_labels, pred, 0))

    return mae_k, cs_k, cs_1

In [31]:
def kfold_cv_aug(model, features_norm, features_aug, labels_norm, labels_aug, k, n_aug, seed):
    """
    Performs stratisfied k-fold cross-validation for augmented data.

    :param model:         The model to train
    :param features_norm: Features of non-augmented images
    :param features_aug:  Features of augmented images
    :param labels_norm:   Labels of the non-augmented features
    :param labels_aug:    Labels of the augmented features
    :param k:             Number of folds for the cross-validation
    :param n_aug:         Number of augmented images per non-augmented image
    :param seed:          Seed for splitting data into test/train sets

    :return: Mean and SD of MAE, CS with alpha = 25 and alpha = 0 years across all folds.
    """
    mae_k = []
    cs_k = []
    cs_1 = []

    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    for train_index, test_index in tqdm.tqdm(kf.split(features_norm, labels_norm)):
        # Getting test and train sets from non-augmented labels and features
        test = [features_norm[idx] for idx in test_index]
        test_labels = [labels_norm[idx] for idx in test_index]

        train = [features_norm[idx] for idx in train_index]
        train_labels = [labels_norm[idx] for idx in train_index]

        # Getting test and training sets from augmented labels and features
        for idx in range(len(features_aug)):
            if int(idx/n_aug) in test_index:
                continue
            else:
                train.append(features_aug[idx])
                train_labels.append(labels_aug[idx])

        # Training the model and predicting dates
        model.fit(train, train_labels)
        pred = model.predict(test)
        pred = [int(i) for i in pred]

        mae_k.append(mae(test_labels, pred))
        cs_k.append(cs(test_labels, pred, 25))
        cs_1.append(cs(test_labels, pred, 0))

    return mae_k, cs_k, cs_1

In [32]:
def test_model(model, train_features, test_features, train_labels, test_labels, plot_bool=0):
    """
    Predicts dates with a given model and displays a scatter plot of 
    the ground truth and predicted dates.

    :param model:          The model to train
    :param train_features: Features to train the model witb
    :param test_features:  Features to test the model with
    :param train_labels:   Labels of train features
    :param test_labels:    Labels of test features
    :param plot_bool:      Boolean if plot should be produced

    :return: Mean Absolute Error (MAE) and Cumulative Score with alpha-values 25 and 0
    """
    model.fit(train_features, train_labels)
    pred = model.predict(test_features)
    pred = [int(i) for i in pred]

    if plot_bool:
        plt.scatter(test_labels, pred)
        plt.ylabel('pred')
        plt.xlabel('true')
        plt.show()
    return round(mae(test_labels, pred), 4), round(cs(test_labels, pred, 25), 4), round(cs(test_labels, pred, 1), 4)

# Feature processing

In [33]:
class FeatType():
    HINGE = 0
    JUNC = 1

class Data():
    def __init__(self, name, feat_dir, feat_aug_dir, k, n_aug):
        self.name = name
        self.feat_dir = feat_dir
        self.feat_aug_dir = feat_aug_dir
        self.k_folds = k
        self.n_aug = n_aug
        if name == "EEA":
            self.test_indices = np.load('./Data/DSS/test_indices.npy')
        elif name == "MPS":
            self.test_indices = np.load('./tfsom/MPS/test_indices.npy')
        elif name == "Himanis":
            self.test_indices_hinge = np.load('./Data/Himanis/Himanis_test_indices_hinge.npy')
            self.test_indices_junclets = np.load('./Data/Himanis/Himanis_test_indices_junclets.npy')
    
    def set_cb_size(self, cb_size):
        self.cb_size = cb_size
    
    def set_kernel(self, kernel):
        self.kernel = kernel

    def set_Cs(self, Cs):
        self.Cs = Cs
    
    def set_Cs_aug(self, Cs):
        self.Cs_aug = Cs

EEA = Data("EEA", "./Data/DSS/features/", "./Data/DSS/features/features_aug_15/", k=4, n_aug=15)
MPS = Data("MPS", "./tfsom/MPS/all/", "./tfsom/MPS/all/", k=10, n_aug=3)
HIMANIS = Data("Himanis", "./Data/Himanis/features/", "./Data/Himanis/features/", k=10, n_aug=10)

In [34]:
def rescale_features(features):
    """
    Rescales features between 0 and 1
    """
    features = np.asarray(features)
    scaler = MinMaxScaler()
    scaler.fit(features)
    data_rescaled = scaler.transform(features)

    return data_rescaled

In [35]:
def rescale_features_split(features1, features2):
    """
    Rescales features between 0 and 1 in the case of 2 split sets of features    
    """
    features1 = np.array(features1)
    features2 = np.array(features2)

    scaler = MinMaxScaler()
    scaler.fit(np.concatenate((features1, features2), axis=0))
    data_rescaled1 = scaler.transform(features1)
    data_rescaled2 = scaler.transform(features2)
    return data_rescaled1, data_rescaled2

In [36]:
def get_features2(feature_dir, hinge_or_junc, dataset):
    """
    Reads feature vectors from files in a directory

    :param feature_dir:   Directory containing feature files
    :param hinge_or_junc: Boolean value whether feature is from Hinge family (0) 
                          or is the Junclets feature (1)
    :param n_aug:         Number of augmented images per non-augmented image

    :return: List of feature vectors and corresponding labels (key years)
    """
    features = []
    labels = []
    count = 0
    nan_count = 0
    print(dataset.n_aug, feature_dir)

    # Iterating over all feature files
    for file in sorted(os.listdir(feature_dir)):
        aug_num = re.search("(_[0-9]?[0-9].p)", file)

        if aug_num is not None:
            aug_num = re.search("([0-9]?[0-9])", aug_num.group())
        if aug_num is None or (aug_num is not None and int(aug_num.group()) <= dataset.n_aug):
            if file != ".DS_Store" and not os.path.isdir(feature_dir + '/' + file):
                f = open(feature_dir + "/" + file)
                # Gets the file's feature vector
                for line in f.readlines():
                    line = line.rstrip().split(" ")
                    if hinge_or_junc == FeatType.HINGE:
                        features.append(line[2:])
                    elif hinge_or_junc == FeatType.JUNC:
                        features.append([float(el) for el in line])
                        count += 1

                    if dataset.name == EEA.name:
                        label = re.search("(-?[0-9][0-9][0-9])", file)
                    elif dataset.name == MPS.name:
                        label = re.search("([0-9][0-9][0-9][0-9])", file) 
                    elif dataset.name == HIMANIS.name:
                        label = re.search("13([0-9][0-9])", file) 

                    
                    labels.append(int(label.group()))
    print(nan_count)

    return features, labels

In [37]:
def get_features(featuredir, size, dataset):
    """
    Returns features and labels in a directory
    """
    if featuredir != 'junclets' and featuredir != 'test/junclets':
        features, labels = get_features2(dataset.feat_dir + featuredir, FeatType.HINGE, dataset)
    elif featuredir == 'junclets':
        prefix = dataset.feat_dir + featuredir + '/size_' + str(size) + '/'
        features, labels = get_features2(prefix, FeatType.JUNC, dataset)
    return features, labels

def get_features_aug(featuredir, size, dataset):
    """
    Returns features and labels in a directory of both non-augmented and augmented images
    """
    if featuredir != 'junclets':
        features_norm, labels_norm = get_features2(dataset.feat_dir + featuredir, FeatType.HINGE, dataset)
        features_aug, labels_aug = get_features2(dataset.feat_aug_dir + featuredir + '_aug', FeatType.HINGE, dataset)
    else:
        suffix = '/size_' + str(size) + '/'
        features_norm, labels_norm = get_features2(dataset.feat_dir + featuredir + suffix, FeatType.JUNC, dataset)
        features_aug, labels_aug = get_features2(dataset.feat_aug_dir + featuredir + '_aug/' + suffix, FeatType.JUNC, dataset)

    return features_norm, labels_norm, features_aug, labels_aug

### Getting indices for test set

In [14]:
# MPS
size_data = 3267
test_size = 0.1 #as a fraction
test_indices = np.random.choice(np.array([i for i in range(size_data)]), int(size_data * test_size), replace=False)
test_idx_path = './test_indices.npy'
if not os.path.exists(test_idx_path):
    np.save(test_idx_path, test_indices)

In [30]:
# Himanis
size_data = 789
test_size = 0.1 #as a fraction
test_indices = np.random.choice(np.array([i for i in range(size_data)]), int(size_data * test_size), replace=False)
test_idx_path = './Data/Himanis/Himanis_test_indices_hinge.npy'
if not os.path.exists(test_idx_path):
    np.save(test_idx_path, test_indices)

In [31]:
# Himanis
size_data = 394
test_size = 0.1 #as a fraction
test_indices = np.random.choice(np.array([i for i in range(size_data)]), int(size_data * test_size), replace=False)
test_idx_path = './Data/Himanis/Himanis_test_indices_junclets.npy'

if not os.path.exists(test_idx_path):
    np.save(test_idx_path, test_indices)

In [32]:
# EA and DSS
idx = 0
for file in sorted(os.listdir('./Data/DSS//DSS_jpg_re/')):
    label = re.search("(-?[0-9][0-9][0-9])", file)
    idx += 1

np.save('./Data/DSS/test_indices.npy', [0, 5, 6, 16, 17])

# Hyper-parameter tuning
Original data

In [ ]:
# For Hinge features
dataset = EEA

feat_names = ['hinge', 'cohinge', 'quadhinge', 'deltahinge', 'tcchinge']

Cs = [pow(2, n) for n in range(-7, 10, 1)]  # hyper-parameter range of values
seeds = [0, 50, 100, 150, 200, 250]

# Iterates over all features
for featuredir in feat_names:
    print(featuredir)

    # Getting the feature vectors and labels
    features, labels = get_features(featuredir, 0, dataset)
    features = rescale_features(features)

    # Getting the test set
    test_indices = dataset.test_indices_hinge if dataset.name == HIMANIS.name else dataset.test_indices
    
    test = [features[idx] for idx in test_indices]
    test_labels = [labels[idx] for idx in test_indices]

    # Getting the train set
    train = []
    train_labels = []
    for idx in range(len(features)):
        if idx not in test_indices:
            train.append(features[idx])
            train_labels.append(labels[idx])

    results = [[0, 0, 0, 0, 0, 0, 0] for _ in range(len(Cs))]
    best_set = {'kernel':'linear', 'C':0, 'MAE':[], 'CS_25':[], 'CS_0':[], 'mean_cs_0':0.0}
    for c_idx in range(len(Cs)):
            results[c_idx][0] = Cs[c_idx]
            res_mae = []
            res_cs_25 = []
            res_cs_0 = []
            for seed in seeds:
                clf = kfold_cv(svm.SVC(kernel='linear', decision_function_shape='ovr', C=Cs[c_idx]), np.array(train), np.array(train_labels), dataset.k_folds, seed)
                res_mae.append(clf[0])
                res_cs_25.append(clf[1])
                res_cs_0.append(clf[2])
                mean_clf = [np.mean(clf[0]), np.std(clf[0]), np.mean(clf[1]), np.std(clf[1]), np.mean(clf[2]), np.std(clf[2])]
                for res_idx in range(len(mean_clf)):
                    results[c_idx][res_idx + 1] += mean_clf[res_idx]
            for res_idx in range(1, len(results[c_idx])):
                    results[c_idx][res_idx] = results[c_idx][res_idx]/len(seeds)
            print(Cs[c_idx], results[c_idx][5], results[c_idx][6])
            accuracy = results[c_idx][5]
    
            if accuracy > best_set['mean_cs_0']:
                best_set['C'] = Cs[c_idx]
                best_set['MAE'] = res_mae
                best_set['CS_25'] = res_cs_25
                best_set['CS_0'] = res_cs_0
                best_set['mean_cs_0'] = accuracy
    print(np.mean(best_set['CS_0']))
            
    if dataset.name == HIMANIS.name:
        np.save('./Data/Himanis/validation_results/' + featuredir + '_validation.npy', best_set)
    if dataset.name == MPS.name:
        np.save('./Data/MPS/validation_results/' + featuredir + '_validation.npy', best_set)
    if dataset.name == EEA.name:
        np.save('./Data/EEA/validation_results/' + featuredir + '_validation.npy', best_set)
        
    print(best_set)
        

Tuning for codebook sizes

In [ ]:

dataset = EEA
featuredir = 'junclets'

Cs = [pow(2, n) for n in range(-7,10, 1)]
seeds = [0, 50, 100, 150, 200, 250]

# Iterates over all sub-codebook sizes
for cb_size in range(5, 35, 5):
    print(cb_size)

    # Getting the feature vectors and labels
    features, labels = get_features(featuredir, cb_size, dataset)
    features = rescale_features(features)

    # Getting the test set
    test_indices = dataset.test_indices_junclets if dataset.name == HIMANIS.name else dataset.test_indices
    test = [features[idx] for idx in test_indices]
    test_labels = [labels[idx] for idx in test_indices]

    # Getting the train set
    train = []
    train_labels = []
    for idx in range(len(features)):
        if idx not in test_indices:
            train.append(features[idx])
            train_labels.append(labels[idx])

    results = [[0, 0, 0, 0, 0, 0, 0] for _ in range(len(Cs))]
    best_set = {'kernel':'linear', 'C':0, 'MAE':[], 'CS_25':[], 'CS_0':[], 'mean_cs_0':0.0}

    # Cross-validation across all seeds and hyper-parameter values
    for c_idx in range(len(Cs)):
            results[c_idx][0] = Cs[c_idx]
            res_mae = []
            res_cs_25 = []
            res_cs_0 = []
            for seed in seeds:
                clf = kfold_cv(svm.SVC(kernel='linear', decision_function_shape='ovr', C=Cs[c_idx]), np.array(train), np.array(train_labels), dataset.k_folds, seed)
                res_mae.append(clf[0])
                res_cs_25.append(clf[1])
                res_cs_0.append(clf[2])
                mean_clf = [np.mean(clf[0]), np.std(clf[0]), np.mean(clf[1]), np.std(clf[1]), np.mean(clf[2]), np.std(clf[2])]
                for res_idx in range(len(mean_clf)):
                    results[c_idx][res_idx + 1] += mean_clf[res_idx]
            for res_idx in range(1, len(results[c_idx])):
                    results[c_idx][res_idx] = results[c_idx][res_idx]/len(seeds)
            print(Cs[c_idx], results[c_idx][5], results[c_idx][6])
            accuracy = results[c_idx][5]
    
            if accuracy > best_set['mean_cs_0']:
                best_set['C'] = Cs[c_idx]
                best_set['MAE'] = res_mae
                best_set['CS_25'] = res_cs_25
                best_set['CS_0'] = res_cs_0
                best_set['mean_cs_0'] = accuracy

    if dataset.name == HIMANIS.name:
        np.save('./Data/Himanis/validation_results/' + featuredir + '_' + str(cb_size) + '_validation.npy', best_set)
    elif dataset.name == MPS.name:
        np.save('./Data/MPS/validation_results/' + featuredir + '_' + str(cb_size) + '_validation.npy', best_set)
    elif dataset.name == EEA.name:
        np.save('./Data/EEA/validation_results/' + featuredir + '_' + str(cb_size) + '_validation.npy', best_set)
    print(best_set)

With augmented data

In [ ]:
feat_names = ['hinge', 'cohinge', 'quadhinge', 'deltahinge', 'tcchinge', 'junclets']
dataset = EEA

EEA.set_cb_size(15)
MPS.set_cb_size(25)
HIMANIS.set_cb_size(10)

Cs = [pow(2, n) for n in range(-7,10, 1)] # range of values for hyper-parameter
seeds = [0, 50, 100, 150, 200, 250]

# Iterates over all features
for featuredir in feat_names:
    print(featuredir)
         
    # Getting feature vectors and labels
    features_norm, labels_norm, features_aug, labels_aug = get_features_aug(featuredir, dataset.cb_size, dataset)
    features_norm, features_aug = rescale_features_split(features_norm, features_aug)

    # Getting test set
    if featuredir == 'junclets':
        test_indices = dataset.test_indices_junclets if dataset.name == HIMANIS.name else dataset.test_indices
    else:
        test_indices = dataset.test_indices_hinge if dataset.name == HIMANIS.name else dataset.test_indices

    test = [features_norm[idx] for idx in test_indices]
    test_labels = [labels_norm[idx] for idx in test_indices]

    # Getting training set from the augmented feature vectors and labels
    train_aug = []
    train_aug_labels = []
    for idx in range(len(features_aug)):
        if int(idx/dataset.n_aug) in test_indices:
            continue
        else:
            train_aug.append(features_aug[idx])
            train_aug_labels.append(labels_aug[idx])

    # Getting training set from the non-augmented feature vectors and labels
    train_norm = []
    train_norm_labels = []
    for idx in range(len(features_norm)):
        if idx not in test_indices:
            train_norm.append(features_norm[idx])
            train_norm_labels.append(labels_norm[idx])
            
    results = [[0, 0, 0, 0, 0, 0, 0] for i in range(len(Cs))]
    best_set = {'kernel':'linear', 'C':0, 'MAE':[], 'CS_25':[], 'CS_0':[], 'mean_cs_0':0.0}

    # Cross-validation across all seeds and hyper-parameter values
    for c_idx in range(len(Cs)):
            results[c_idx][0] = Cs[c_idx]
            res_mae = []
            res_cs_25 = []
            res_cs_0 = []
            for seed in seeds:
                clf = kfold_cv_aug(svm.SVC(kernel='linear', decision_function_shape='ovr', C=Cs[c_idx]), 
                                   np.array(train_norm), np.array(train_aug), 
                                   np.array(train_norm_labels), np.array(train_aug_labels), 
                                   dataset.k_folds, 
                                   dataset.n_aug,
                                   seed)
                res_mae.append(clf[0])
                res_cs_25.append(clf[1])
                res_cs_0.append(clf[2])
                mean_clf = [np.mean(clf[0]), np.std(clf[0]), np.mean(clf[1]), np.std(clf[1]), np.mean(clf[2]), np.std(clf[2])]
                for res_idx in range(len(mean_clf)):
                    results[c_idx][res_idx + 1] += mean_clf[res_idx]
            for res_idx in range(1, len(results[c_idx])):
                    results[c_idx][res_idx] = results[c_idx][res_idx]/len(seeds)
            print(Cs[c_idx], results[c_idx][5], results[c_idx][6])
            accuracy = results[c_idx][5]
    
            if accuracy > best_set['mean_cs_0']:
                best_set['C'] = Cs[c_idx]
                best_set['MAE'] = res_mae
                best_set['CS_25'] = res_cs_25
                best_set['CS_0'] = res_cs_0
                best_set['mean_cs_0'] = accuracy
    if dataset.name == HIMANIS.name:
        np.save('./Data/Himanis/validation_results/' + featuredir + '_validation_aug.npy', best_set)
    elif dataset.name == MPS.name:
        np.save('./Data/MPS/validation_results/' + featuredir + '_validation_aug.npy', best_set)
    elif dataset.name == EEA.name:
        np.save('./Data/EEA/validation_results/' + featuredir + '_validation_aug.npy', best_set)
    print(best_set)

In [ ]:
dataset = EEA
file_name = './Data/' + dataset.name + '/validation_results/validation_aug.csv'


with open(file_name, mode='w') as file:
    if dataset == MPS.name:
        file.write('feature, Mean MAE, MAE SD, Mean CS(=25), CS(=25) SD, Mean CS(=0), CS(=0) SD\n')
    else:
        file.write('feature, Mean MAE, MAE SD, Mean CS(=0), CS(=0) SD\n')

    for featuredir in ['hinge', 'cohinge', 'quadhinge', 'deltahinge', 'tcchinge', 'junclets']:
        print(featuredir)
        results_dict = np.load('./Data/' + dataset.name + '/validation_results/' + featuredir + '_validation_aug.npy', allow_pickle=True)
        results_dict = results_dict.item()
        mae_folds = results_dict['MAE']
        cs_0_folds = results_dict['CS_0']
        
        print(np.mean(mae_folds), np.std(mae_folds), np.mean(cs_0_folds), np.std(cs_0_folds))
        
        mae_mean = round(np.mean(mae_folds), 2)
        mae_sd = round(np.std(mae_folds), 2)
        cs_0_mean = round(np.mean(cs_0_folds), 2)
        cs_0_sd = round(np.std(cs_0_folds), 2)

        if dataset.name == MPS.name:
            cs_25_folds = results_dict['CS_25']
            cs_25_mean = round(np.mean(cs_25_folds), 2)
            cs_25_sd = round(np.std(cs_25_folds), 2)
            print(cs_25_sd)
            to_write = featuredir + ',' + str(mae_mean) + ',' + str(mae_sd) + ',' + str(cs_25_mean) + ',' + str(cs_25_sd) + ',' + str(cs_0_mean) + ',' + str(cs_0_sd) + '\n'
        else:
            to_write = featuredir + ',' + str(mae_mean) + ',' + str(mae_sd) + ',' + str(cs_0_mean) + ',' + str(cs_0_sd) + '\n'
        
        file.write(to_write)



In [ ]:
featuredir = 'junclets'
dataset = EEA

file_name = './Data/' + dataset.name + '/validation_results/validation_junclets.csv'

with open(file_name, mode='w') as file:
    if dataset == MPS.name:
        file.write('subcodebook size, Mean MAE, MAE SD, Mean CS(=25), CS(=25) SD, Mean CS(=0), CS(=0) SD\n')
    else:
        file.write('subcodebook size, Mean MAE, MAE SD, Mean CS(=0), CS(=0) SD\n')
    for cb_size in range(5, 35, 5):
        print(cb_size)
        results_dict = np.load('./Data/' + dataset.name + '/validation_results/' + featuredir + '_' + str(cb_size) + '_validation.npy', allow_pickle=True)
        results_dict = results_dict.item()
        mae_folds = results_dict['MAE']
        cs_0_folds = results_dict['CS_0']
        
        print(np.mean(mae_folds), np.std(mae_folds), np.mean(cs_0_folds), np.std(cs_0_folds))
        
        mae_mean = round(np.mean(mae_folds), 2)
        mae_sd = round(np.std(mae_folds), 2)
        cs_0_mean = round(np.mean(cs_0_folds), 2)
        cs_0_sd = round(np.std(cs_0_folds), 2)

        if dataset.name == MPS.name:
            cs_25_folds = results_dict['CS_25']
            cs_25_mean = round(np.mean(cs_25_folds), 2)
            cs_25_sd = round(np.std(cs_25_folds), 2)
            print(cs_25_sd)
            to_write = str(cb_size) + ',' + str(mae_mean) + ',' + str(mae_sd) + ',' + str(cs_25_mean) + ',' + str(cs_25_sd) + ',' + str(cs_0_mean) + ',' + str(cs_0_sd) + '\n'
        else:
            to_write = str(cb_size) + ',' + str(mae_mean) + ',' + str(mae_sd) + ',' + str(cs_0_mean) + ',' + str(cs_0_sd) + '\n'
        
        file.write(to_write)

# Testing

With original data

In [ ]:
feat_names = ['hinge', 'cohinge', 'quadhinge', 'deltahinge', 'tcchinge', 'junclets']

EEA.set_kernel('linear')
EEA.set_Cs([1, 1, 1, 0.125, 0.125, 0.03125])
MPS.set_kernel('linear')
MPS.set_Cs([8, 0.0625, 0.125, 1, 1, 0.0625])
HIMANIS.set_kernel('linear')
HIMANIS.set_Cs([4, 0.03125, 0.25, 0.5, 1, 0.125])
HIMANIS.set_cb_size(10)

dataset = EEA

idx_c = 0

file_name = './Data/' + dataset.name + '/validation_results/test.csv'

with open(file_name, mode='w') as file:
    file.write('feature,MAE,CS(=0)\n')
    for featuredir in feat_names:
        print(featuredir)
        # Getting feature vectores + labels
        features, labels = get_features(featuredir, dataset.cb_size, dataset)
        features = rescale_features(features)

        # Getting test set
        if featuredir == 'junclets':
            test_indices = dataset.test_indices_junclets if dataset.name == HIMANIS.name else dataset.test_indices
        else:
            test_indices = dataset.test_indices_hinge if dataset.name == HIMANIS.name else dataset.test_indices

        test = [features[idx] for idx in test_indices]
        test_labels = [labels[idx] for idx in test_indices]

        # Getting train set
        train = []
        train_labels = []
        for idx in range(len(features)):
            if idx not in test_indices:
                train.append(features[idx])
                train_labels.append(labels[idx])

        # Date prediction of test set
        print(idx_c, dataset.Cs[idx_c])
        mae_res, cs_res25, cs_res1 = test_model(svm.SVC(kernel=dataset.kernel, decision_function_shape='ovr', C=dataset.Cs[idx_c]), train, test, train_labels, test_labels, plot_bool=1)
        print("MAE: %.4f  \t CS (=25): %.4f  \t CS(=1): %.4f " % (mae_res, cs_res25, cs_res1))
        print("%.4f,%.4f,%.4f" % (mae_res, cs_res25, cs_res1))

        to_write = featuredir + ',' + str(round(mae_res, 2)) + ',' + str(round(cs_res1, 2)) + '\n'
        file.write(to_write)

        idx_c += 1

With augmented data

In [ ]:
feat_names = ['hinge', 'cohinge', 'quadhinge', 'deltahinge','tcchinge', 'junclets']

EEA.set_Cs_aug([1, 1, 1, 2, 0.25, 0.25])
MPS.set_Cs_aug([2, 0.0625, 0.0625, 1, 1, 0.0625])
HIMANIS.set_Cs_aug([0.125, 0.03125, 0.125, 64, 1, 0.0625])

dataset = HIMANIS

idx_c = 0
file_name = './Data/' + dataset.name + '/validation_results/test_aug.csv'

with open(file_name, mode='w') as file:
    for featuredir in feat_names:
        print(featuredir)
        # Getting feature vectors + labels
        features_norm, labels_norm, features_aug, labels_aug = get_features_aug(featuredir, dataset.cb_size, dataset)
        features_norm, features_aug = rescale_features_split(features_norm, features_aug)

        # Test set from non-augmented data
        if featuredir == 'junclets':
            test_indices = dataset.test_indices_junclets if dataset.name == HIMANIS.name else dataset.test_indices
        else:
            test_indices = dataset.test_indices_hinge if dataset.name == HIMANIS.name else dataset.test_indices

        test = [features_norm[idx] for idx in test_indices]
        test_labels = [labels_norm[idx] for idx in test_indices]
        
        # Train set augmented data
        train_aug = []
        train_aug_labels = []
        for idx in range(len(features_aug)):
            if int(idx/dataset.n_aug) in test_indices:
                continue
            else:
                train_aug.append(features_aug[idx])
                train_aug_labels.append(labels_aug[idx])

        # Train set non-augmented data
        train_norm = []
        train_norm_labels = []
        for idx in range(len(features_norm)):
            if idx not in test_indices:
                train_norm.append(features_norm[idx])
                train_norm_labels.append(labels_norm[idx])

        # concatenating augmented and non-augmented train sets
        train = train_norm + train_aug
        train_labels = train_norm_labels + train_aug_labels
        print(len(train), len(train_labels), len(train_aug))

        # Date prediction on test set
        mae_res, cs_res25, cs_res1 = test_model(svm.SVC(kernel=dataset.kernel, decision_function_shape='ovr', C=dataset.Cs_aug[idx_c]), train, test, train_labels, test_labels, plot_bool=1)
        print("MAE: %.4f  \t CS (=25): %.4f  \t CS(=1): %.4f " % (mae_res, cs_res25, cs_res1))
        print("%.4f,%.4f,%.4f" % (mae_res, cs_res25, cs_res1))

        to_write = featuredir + ',' + str(round(mae_res, 2)) + ',' + str(round(cs_res1, 2)) + '\n'
        file.write(to_write)

        idx_c += 1

## ANOVA 

In [ ]:

dataset = EEA

for featuredir in ['hinge', 'cohinge', 'quadhinge', 'deltahinge', 'tcchinge', 'junclets']:
    print(featuredir)
    if featuredir == 'junclets':
        results_dict = np.load('./Data/' + dataset.name + '/validation_results/' + featuredir + '_' + str(dataset.cb_size) + '_validation.npy', allow_pickle=True)
        results_dict = results_dict.item()
    else:
        results_dict = np.load('./Data/' + dataset.name + '/validation_results/' + featuredir + '_validation.npy', allow_pickle=True)
        results_dict = results_dict.item()

    results_aug_dict = np.load('./Data/' + dataset.name + '/validation_results/' + featuredir + '_validation_aug.npy', allow_pickle=True)
    results_aug_dict = results_aug_dict.item()

    mae = np.asarray(results_dict['MAE'])
    mae_aug = np.asarray(results_aug_dict['MAE'])

    cs_0 = np.asarray(results_dict['CS_0'])
    cs_0_aug = np.asarray(results_aug_dict['CS_0'])

    anova_mae = f_oneway(mae.flatten(), mae_aug.flatten())
    anova_cs0 = f_oneway(cs_0.flatten(), cs_0_aug.flatten())

    print("MAE: ", anova_mae)
    print("CS (alpha = 0): ", anova_cs0)

    if dataset.name == MPS.name:
        cs_25 = np.asarray(results_dict['CS_25'])
        cs_25_aug = np.asarray(results_aug_dict['CS_25'])
        anova_cs25 = f_oneway(cs_25.flatten(), cs_25_aug.flatten())
        print("CS (alpha = 25): ", anova_cs25)


    